In [1]:
test = '''
contract get_fib_n(n : int){

    let x: int = 0;

    entry sum(n: int){
        let a: int = 2;
        let b: int = 3;
        x = a + b;
    }
}
'''

In [2]:
from lexer.sly_lexer import TzScriptLexer, process_lexer_tokens 
from lexer.lex_token import Token
from parser.tzscript_grammar import TZSCRIPT_GRAMMAR,idx, num, typex, contract, ifx, elsex,equal, plus, minus, star, div,semi, colon, comma, dot, opar, cpar, ocur, ccur,let, func,entry,lessthanequal,greaterthanequal, iniquelaty, lessthan,greaterthan,equalequal, returnx
from parser.slr_parser import SLR1Parser, build_slr_ast

lexer = TzScriptLexer()
lexer_tokens = list(lexer.tokenize(test))
tokens = process_lexer_tokens(lexer_tokens)

parser = SLR1Parser(TZSCRIPT_GRAMMAR, verbose=True)

terminals = [token.token_type for token in tokens]
derivation = parser(terminals, True)

0 	 S' -> .<program>, 
	 <program> -> .contractid(<param-list>){<stat_list>},  

1 	 S' -> <program>.,  

2 	 <program> -> contract.id(<param-list>){<stat_list>},  

3 	 <program> -> contractid.(<param-list>){<stat_list>},  

4 	 <param-list> -> .<param>,<param-list>, 
	 <program> -> contractid(.<param-list>){<stat_list>}, 
	 <param-list> -> .<param>, 
	 <param> -> .id:type,  

5 	 <param-list> -> <param>.,<param-list>, 
	 <param-list> -> <param>.,  

6 	 <param-list> -> .<param>,<param-list>, 
	 <param-list> -> .<param>, 
	 <param> -> .id:type, 
	 <param-list> -> <param>,.<param-list>,  

7 	 <param> -> id.:type,  

8 	 <param> -> id:.type,  

9 	 <param> -> id:type.,  

10 	 <param-list> -> <param>,<param-list>.,  

11 	 <program> -> contractid(<param-list>.){<stat_list>},  

12 	 <program> -> contractid(<param-list>).{<stat_list>},  

13 	 <stat_list> -> .<stat>, 
	 <stat> -> .<while-stat>, 
	 <def-entry> -> .entryid(<param-list>){<stat_list>}, 
	 <let-var>> -> .letid:type=<expr>;, 

In [3]:
productions, operations = derivation


In [4]:
ast = build_slr_ast(productions, operations, tokens)

In [5]:
from visitors.string_rep_visitor import FormatVisitor
format = FormatVisitor()
format.visit(ast)

'\\__ProgramNode: contract node.idx(n : int) [<stat>; ... <stat>;]\n\t\\__AttrDeclarationNode: n : int\n\t\t\\__VarDeclarationNode: let x = <expr> : int\n\t\t\t\\__ ConstantNumNode: 0\n\t\t\\__EntryDeclarationNode: Entry sum(n : int)\n\t\t\t\\__AttrDeclarationNode: n : int\n\t\t\t\t\\__VarDeclarationNode: let a = <expr> : int\n\t\t\t\t\t\\__ ConstantNumNode: 2\n\t\t\t\t\\__VarDeclarationNode: let b = <expr> : int\n\t\t\t\t\t\\__ ConstantNumNode: 3\n\t\t\t\t\\__VarCallNode: x = <expr>\n\t\t\t\t\t\\__<expr> PlusNode <expr>\n\t\t\t\t\t\t\\__ VariableNode: a\n\t\t\t\t\t\t\\__ VariableNode: b'

In [6]:
from visitors.high_level_ir_generator_visitor import TzScriptToHighLevelIrVisitor
high_level_ir = TzScriptToHighLevelIrVisitor()

In [7]:
from visitors.hl_string_repre import HLReprVisitor
hl_repr = HLReprVisitor()
ir = high_level_ir.visit(ast)

mirame
mirame


In [8]:
print(hl_repr.visit(ir))

contract { entrypoint{...} storage{...} code{...}} 
	entrypoint  { <id,params> , <id,params> , ... , <id,params> }
		\__EntryPointDeclarationNode: sum ([<parser.tzscript_ast.AttrDeclarationNode object at 0x7f50f6824510>])
	storage { <id,type> , <id,type> , ... , <id,type> }
		\__StorageDeclarationNode: x : int
	code { <stat> , <stat> , ... , <stat> }
		\__IfEntryNode: if sum then  else 
			\__PushValueNode: push num 2
			\__PushVariableNode: push int a
			\__PushValueNode: push num 3
			\__PushVariableNode: push int b


In [9]:
for x in ir.entrypoints.entrypoint_list:
    print(x.id)

sum
